#### Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

Conversational RAG: Enable a chatbot experience over an external source of data
Agents: Build a chatbot that can take actions
This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv() ## loading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [2]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7f8e905e98d0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7f8e905e8c10>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage    
model.invoke(([HumanMessage(content="Hi , My name is Nidhi and I am a Chief AI Engineer")]))


AIMessage(content="Hello Nidhi, \n\nIt's a pleasure to meet you! That's a fascinating role. \n\nWhat kind of projects are you working on as a Chief AI Engineer?  I'm always interested in learning more about how AI is being used in different fields.\n", response_metadata={'token_usage': {'completion_tokens': 60, 'prompt_tokens': 28, 'total_tokens': 88, 'completion_time': 0.1239762, 'prompt_time': 0.002626192, 'queue_time': None, 'total_time': 0.126602392}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-72c28beb-1861-43eb-ad33-2c3c6369c7ab-0', usage_metadata={'input_tokens': 28, 'output_tokens': 60, 'total_tokens': 88})

In [4]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Nidhi and I am a Chief AI Engineer"),
        AIMessage(content="Hello Nidhi, \n\nIt's a pleasure to meet you! That's a fascinating role. \n\nWhat kind of projects are you working on as a Chief AI Engineer?  I'm always interested in learning more about how AI is being used in different fields.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You said your name is Nidhi and that you are a Chief AI Engineer!  \n\nIs there anything else you'd like to tell me about yourself or your work?  I'm eager to learn more. 😊  \n\n", response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 108, 'total_tokens': 158, 'completion_time': 0.103034643, 'prompt_time': 0.004974634, 'queue_time': None, 'total_time': 0.108009277}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-9b2a77f3-1142-4fe1-ae3a-0f55a6ff7e9f-0', usage_metadata={'input_tokens': 108, 'output_tokens': 50, 'total_tokens': 158})

#### Message History¶
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [6]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)-> BaseChatMessageHistory:   # this is will imported from Basechatmessagehistory
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [7]:
config = {"configurable":{"session_id":"chat1"}}

In [10]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Nidhi and I am a Chief AI Engineer")],
    config=config
)

Error in RootListenersTracer.on_chain_end callback: ValueError()


Error in callback coroutine: ValueError()


In [11]:
response.content

"Hello Nidhi! It's great to meet you. \n\nBeing a Chief AI Engineer is a very exciting role. What kind of work are you doing in AI? \n\nAre you focused on a particular industry or application? I'm always interested in learning about the innovative ways people are using AI.\n"

In [12]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


AIMessage(content='Your name is Nidhi.  \n\nI remember from our earlier conversation. 😊  How can I help you today? \n\n', response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 198, 'total_tokens': 227, 'completion_time': 0.058965801, 'prompt_time': 0.008109866, 'queue_time': None, 'total_time': 0.067075667}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-a63437fe-b7ab-4cbf-872c-ccc7b79ab70a-0', usage_metadata={'input_tokens': 198, 'output_tokens': 29, 'total_tokens': 227})

In [13]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'd be happy to use it! 😊\n"

In [14]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is Sri")],
    config=config1
)
response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


"Hi Sri! It's nice to meet you. 😊 \n\nIs there anything I can help you with today?\n"

In [15]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


'Your name is Sri!  \n\nI remember that you told me earlier. 😊  \n\n'

#### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [16]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [17]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Nidhi")]})

AIMessage(content="Hello Nidhi,\n\nIt's nice to meet you! How can I help you today? 😊  \n", response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 37, 'total_tokens': 62, 'completion_time': 0.050513076, 'prompt_time': 0.002972567, 'queue_time': None, 'total_time': 0.053485643}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-a2e3b227-bdca-4e44-b201-192daf723b47-0', usage_metadata={'input_tokens': 37, 'output_tokens': 25, 'total_tokens': 62})

In [18]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [19]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Nidhi")],
    config=config
)

response

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


AIMessage(content="Hello Nidhi, it's nice to meet you! \n\nHow can I help you today? 😊  \n\n", response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 37, 'total_tokens': 64, 'completion_time': 0.054732868, 'prompt_time': 0.003317344, 'queue_time': None, 'total_time': 0.058050212}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-f6f9e248-0f3e-4be3-8d30-265b928d1dfa-0', usage_metadata={'input_tokens': 37, 'output_tokens': 27, 'total_tokens': 64})

In [20]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


'Your name is Nidhi. 😊  I remember! \n\n\n'

In [21]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [25]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is nidhi")],"language":"Hindi"})
response.content

'नमस्ते निधि! 😊 \n\nआपका परिचय जानकर अच्छा लगा। मैं आपकी सहायता करने के लिए तैयार हूँ। आप मुझसे क्या पूछना चाहेंगे? \n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [26]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [30]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Nidhi")],"language":"Hindi"},
    config=config
)
repsonse.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


'¡Hola Nidhi! ¿Cómo estás? 😊 \n'

In [28]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


In [29]:
response.content

'आपका नाम निधि है। 😊  \n'

#### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in. 'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [31]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

/Users/Nidhi/Desktop/Generative AI/LangChain/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/Nidhi/Desktop/Generative AI/LangChain/venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [32]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As an AI, I don't have access to your personal information, including your ice cream preferences! \n\nWhat's your favorite flavor?  🍦😊\n"

In [33]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "what\'s 2 + 2" 😊  \n\n\n\nIs there anything else I can help you with?\n'

In [34]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [35]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


"As an AI, I don't have memory of past conversations or personal information about you, so I don't know your name.  What's your name? 😊  \n\n"

In [36]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


"As a large language model, I have no memory of past conversations. If you'd like to ask me a math problem, I'm happy to help! 😊  Just let me know what it is. \n\n"